# **EDA (Análisis Exploratorio de Datos)**

In [823]:
# importar bibliotecas:
import pandas as pd
import re
import numpy as np
import math # tratamiento de nulos
from datetime import datetime
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import ast


## **1- COMPRENSIÓN DEL NEGOCIO**

**Observaciones y conclusiones:**

Steam es una plataforma digital de distribución de videojuegos desarrollada por la empresa Valve Corporation. Fue lanzada en 2003 y se ha convertido en una de las plataformas más populares para comprar, descargar y jugar videojuegos en computadoras personales.

Steam ofrece una amplia variedad de funciones y servicios relacionados con los videojuegos:

1. **Distribución Digital:** Steam permite a los usuarios comprar y descargar videojuegos directamente a sus computadoras. Los juegos se vinculan a las cuentas de los usuarios y se pueden descargar y jugar en cualquier momento.

2. **Actualizaciones Automáticas:** Steam proporciona actualizaciones automáticas para los juegos comprados. Esto asegura que los juegos estén siempre actualizados con los últimos parches y mejoras.

3. **Comunidad:** Steam cuenta con una comunidad en línea donde los jugadores pueden interactuar, unirse a grupos, chatear con amigos, compartir capturas de pantalla y logros, y más.

4. **Tienda:** La tienda de Steam ofrece una amplia selección de juegos de diferentes géneros y desarrolladores. Los jugadores pueden buscar juegos, leer reseñas, ver trailers y comprar títulos directamente desde la plataforma.

5. **Biblioteca de Juegos:** Los juegos adquiridos se almacenan en la biblioteca de juegos del usuario, lo que facilita la descarga y el acceso.

6. **Steam Workshop:** Muchos juegos en Steam tienen soporte para el Steam Workshop, una plataforma que permite a los jugadores crear, compartir y descargar contenido adicional, como mods, mapas y elementos creados por la comunidad.

7. **Steam Cloud:** Steam ofrece almacenamiento en la nube para guardar partidas y configuraciones de juego, lo que permite a los jugadores acceder a su progreso desde diferentes dispositivos.

8. **Ofertas y Promociones:** Steam es conocido por sus ofertas regulares, ventas y promociones que permiten a los jugadores comprar juegos a precios reducidos.

En resumen, Steam es una plataforma integral que facilita la adquisición, descarga y gestión de videojuegos en PC. Ha tenido un impacto significativo en la industria de los videojuegos y ha influido en cómo se distribuyen y consumen los juegos en la era digital.

### **1-1- OBTENER DATOS PROVENIENTES DE ETL**

In [824]:
# Lectura de los CSV en un df respectivamente
ruta = "etl_games.csv"
df_games = pd.read_csv(ruta, low_memory=False)

In [825]:
# Lectura de los CSV en un df respectivamente
ruta = "etl_reviews.csv"
df_reviews = pd.read_csv(ruta, low_memory=False)

In [826]:
# Lectura de los CSV en un df respectivamente
ruta = "etl_items.csv"
df_items = pd.read_csv(ruta, low_memory=False)

#### **1-1- df_games - checkpoint**

In [827]:
# Creación punto de restauración
df_games_1_1 = df_games.copy()

In [828]:
# Carga punto de restauración
df_games = df_games_1_1.copy()

#### **1-1- df_reviews - checkpoint**

In [829]:
# Creación punto de restauración
df_reviews_1_1 = df_reviews.copy()

In [830]:
# Carga punto de restauración
df_reviews = df_reviews_1_1.copy()

#### **1-1- df_items - checkpoint**

In [831]:
# Creación punto de restauración
df_items_1_1 = df_items.copy()

In [832]:
# Carga punto de restauración
df_items = df_items_1_1.copy()

### **1-2- EXAMINAR ESTRUCTURA GENERAL**

- Variables relevantes e irrelevantes

**Observaciones y conclusiones:**

- Dadas las dimensiones y el peso de los datasets games, items y reviews se planea generar nuevos datasets que contenga cada uno la información específica para ejecutar las consultas que demanda el proyecto. De esta manera se trabajarán datasets de dimensiones más pequeñas con menor tamaño de almacenamiento para agilizar las consultas.

- Los datasets resultantes se listan a continuación incluyendo las columnas (con una breve obervacion y tipo de dato), el dataset de origen de cada columna y la consulta (función) que se le va a realizar:

    1. df_userdata = def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
        <br>
        - df_items.user_id = (str)
        - df_items.item_id = (str)
        - df_items.items_count = (int)
        - df_games.price = (float)
        <br>
        <br>
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - user_id (str) = id para cada usuario
        - money_spent (float) = dinero gastado por cada usuario
        - total_recommend (float) = recomendaciones totales por cada usuario
        - items_count (int) = cantidad total de items por cada usuario
        <br>
        <br>
    2. df_countreviews = def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.
        <br>
        - df_reviews.user_id = (str)
        - df_reviews.posted_date = (datetime)
        - df_reviews.recommend = (bool)
        <br>
        <br>
        Observaciones:
        - Cada usuario puede tener solo 1 review por item y aunque pueda editar esa review en repetidas ocasiones igualmente será tenida en cuenta como 1 sola.
        - Cantidad de usuarios que publicaron reviews entre esas fechas. NO es cantidad de reviews publicadas en esa fecha
        - Porcentaje de recomendación de los True respecto a toda la cantidad de publicaciones entre esas fechas
        <br>
        <br>
    3. df_genre = def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.
        <br>
        - df_items.item_id = (str)
        - df_items.playtime_hours = (int)
        - df_games.genres = (str / list)
        <br>
        <br>
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - genre (str) = genero unico dentro de la lista total de generos
        - ranking (int) = puesto que ocupa cada género de acuerdo a PlayTimeForever
        <br>
        <br>
    4. df_userforgenre = def userforgenre( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.
        <br>
        - df_items.user_id = (str)
        - df_items.playtime_mins = (int)
        - df_games.genres = (str / list)
        - df_items.user_url = (str)
        <br>
        <br>
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - genre (str) = genero unico dentro de la lista total de generos
        - ranking (int) = posición de cada usuario segun la mayor cantidad de horas de juego
        - playtime_hours (int) = cantidad de horas de juego totales
        - user_id (str) = identificacion de usuario
        - user_url (str) = url de usuario
        <br>
        <br>
    5. df_developer = def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.
        <br>
        - df_games.release_date (extraer el año y convertir en "release_year")(int)
        - df_games.price = (float)
        - df_games.item_id = (str)
        - df_games.developer = (str)
        <br>
        <br>
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - developer (str) = desarrollador
        - release_year(str) = año de lanzamiento de item
        - items_count (int) = cant de items lanzados por año
        - free_count (int) = cant items price = 0 por año
        - free_percent (float) = porcentaje de free respecto al total de items del año
        <br>
        <br>
    6. df_sentiment_analysis = def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
        <br>
        - df_games.release_year = (extraer el año y convertir en "release_year")(int)
        - df_games.item_id = (str)
        - df_reviews.review = (transformar la columna en "sentiment_analysis")(int)
        <br>
        <br>
        A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
        - release_year (str) = año de lanzamiento
        - sentiment_label (str) = analisis de sentimiento
        - reviews_count (int) = se refiere a cantidad de reseñas (NO tienen que ser reseñas válidas)
        <br>
        <br>
- Por esta razón se seleccionarán únicamente las columnas necesarias de los datasets games, items y reviews con el fin de realizar las respectivas combianciones para obtener como resultado los datasets deseados


#### **1-2- df_games**

**Observaciones y conclusiones:**

- Corresponde a un dataset que indica los juegos disponibles en la plataforma de steam y sus respectivas características
- A primera vista se van a descartar la siguientes columnas que se consideran innecesarias para las demandas del proyecto como son:
    - publisher
    - title
    - url
    - reviews_url
    - early_access

In [833]:
# Carga pto restauración
df_games = df_games_1_1.copy()

In [834]:
# Examinar tamaño del archivo y dimensiones del dataset
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB


In [835]:
# Eliminar columnas en dataset
col_to_drop = ["publisher", "title", "url", "reviews_url", "early_access"]
df_games.drop(columns=col_to_drop, inplace=True)

In [836]:
# Verificar peso y dimensiones del dataframe
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   app_name      32133 non-null  object 
 2   release_date  30068 non-null  object 
 3   tags          31972 non-null  object 
 4   specs         31465 non-null  object 
 5   price         30758 non-null  object 
 6   id            32133 non-null  float64
 7   developer     28836 non-null  object 
dtypes: float64(1), object(7)
memory usage: 2.0+ MB


##### **1-2- df_games - checkpoint**

In [837]:
# Creación punto de restauración
df_games_1_2 = df_games.copy()
df_games_1_2 = df_games_1_2.reset_index(drop=True)    # Restablecer el índice

In [838]:
# Carga punto de restauración
df_games = df_games_1_2.copy()

#### **1-2- df_reviews**

**Observaciones y conclusiones:**

- Corresponde a un dataset que indica las reseñas que los jugadores hacen de los videojuegos y las características relacionadas con estas reseñas
- A primera vista se van a descartar la siguientes columnas que se consideran innecesarias para las demandas del proyecto como son:
    - user_url
    - funny
    - helpful

In [839]:
# Carga pto restauración
df_reviews = df_reviews_1_1.copy()

In [840]:
# Examinar tamaño del archivo y dimensiones del dataset
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        8151 non-null   object
 3   posted       59305 non-null  object
 4   last_edited  6140 non-null   object
 5   item_id      59305 non-null  int64 
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59275 non-null  object
dtypes: bool(1), int64(1), object(7)
memory usage: 3.7+ MB


In [841]:
# Eliminar columnas en dataset
col_to_drop = ["user_url", "funny", "helpful", "last_edited"]
df_reviews.drop(columns=col_to_drop, inplace=True)

In [842]:
# Verificar peso y dimensiones del dataframe
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   posted     59305 non-null  object
 2   item_id    59305 non-null  int64 
 3   recommend  59305 non-null  bool  
 4   review     59275 non-null  object
dtypes: bool(1), int64(1), object(3)
memory usage: 1.9+ MB


##### **1-2- df_reviews - checkpoint**

In [843]:
# Creación punto de restauración
df_reviews_1_2 = df_reviews.copy()
df_reviews_1_2 = df_reviews_1_2.reset_index(drop=True)    # Restablecer el índice

In [844]:
# Carga punto de restauración
df_reviews = df_reviews_1_2.copy()

#### **1-2- df_items**

**Obervaciones y conclusiones:**

- Corresponde a un dataset que indica las características de cada usuario con respecto a los videojuegos que posee
- A primera vista se van a descartar la siguientes columnas que se consideran innecesarias para las demandas del proyecto como son:
    - steam_id
    - item_name
    - playtime_2weeks

In [845]:
# Carga pto restauración
df_items = df_items_1_1.copy()

In [846]:
# Examinar tamaño del archivo y dimensiones del dataset
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          int64 
 3   user_url          object
 4   item_id           int64 
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(5), object(3)
memory usage: 314.5+ MB


In [847]:
# Eliminar columnas en dataset
col_to_drop = ["steam_id", "item_name", "playtime_2weeks"]
df_items.drop(columns=col_to_drop, inplace=True)

In [848]:
# Verificar peso y dimensiones del dataframe
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   user_url          object
 3   item_id           int64 
 4   playtime_forever  int64 
dtypes: int64(3), object(2)
memory usage: 196.6+ MB


##### **1-2- df_items - checkpoint**

In [849]:
# Creación punto de restauración 
df_items_1_2 = df_items.copy()
df_items_1_2 = df_items_1_2.reset_index(drop=True)    # Restablecer el índice

In [850]:
# Carga punto de restauración 
df_items = df_items_1_2.copy()

## **2- LIMPIEZA DE DATOS**

- Eliminar duplicados
- Examinar columnas:
    - Tratamiento de faltantes y nulos
    - Estandarizar datos: tipo de variable vs. tipo de dato
    - Revisar sintaxis
    - Ajustar títulos
    - Estadística descriptiva
        - Revisión estadística (gráficos de apoyo)
        - Identificar valores extremos: tratar atípicos, outliers

**Observaciones y conclusiones:**

- Se realizarán las transformaciones de cada una de las columnas de los datasets games, reviews e items de acuerdo al tipo de variable que corresponde

### **2- Funciones útiles**

In [851]:
# Revisar la cantidad de tipos de datos y valores nulos por cada columna de un dataframe

def count_column_types(df):
    '''
    Obtener los tipos de datos unicos y sus respectivos recuentos para cada columna.
    Funcion que recibe como parámetro un dataframe
    '''
    
    result = df.apply(lambda col: col.apply(type) .value_counts())          # Aplicar la función a todas las columnas del DataFrame
    result = result.T.reset_index()                                         # Transponer el resultado y restablecer el índice
    result. columns = ["Column"] + result.columns[1:].tolist()              # Renombrar las columnas

    serie_nulos = df.isnull().sum()                                         # Hallamos los valores nulos y se almacenan en una serie
    serie_nulos = serie_nulos.reset_index()                                 # Convierte el índice en una columna
    serie_nulos.columns = ['Column', 'Nulos']                               # Asigna un nombre a la columna del índice si es necesario

    df_1 = result
    df_2 = serie_nulos
    columna_union = "Column"
    df_merge = df_1.merge(df_2, on=columna_union, how='left')               # Realizar un left join entre df_userdata y df_games

    df_merge["Nulos %"] = round(df_merge["Nulos"] / df.shape[0] * 100, 2)   # Porcentaje total de registros nulos

    print("\nfilas completamente nulas: ", df.isna().all(axis=1).sum())     # Filas que se encuentran totalmente en nulo

    return df_merge

In [852]:
# Revisar los valores unicos de cada columna para estandarizar

def verificar_datos_unicos(df):
    '''
    Función para la cantidad de valores únicos y cuáles son estos para cada columna.
    Recibe como parámetro el dataframe a examinar.

    '''
    columnas_df = df.columns.tolist()
    tipos_datos = [float, int, str]

    for columna in columnas_df:
        for tipo in tipos_datos:
            filtro = df[columna][df[columna].apply(lambda x: isinstance(x, tipo))]
            valores_unicos = filtro.unique()
            print(columna, " (", tipo.__name__, ") ", len(valores_unicos),": ", valores_unicos)
        print("")

### **2- df_games**

- (Duplicados, nulos, formato columnas)<br>

In [853]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   app_name      32133 non-null  object 
 2   release_date  30068 non-null  object 
 3   tags          31972 non-null  object 
 4   specs         31465 non-null  object 
 5   price         30758 non-null  object 
 6   id            32133 non-null  float64
 7   developer     28836 non-null  object 
dtypes: float64(1), object(7)
memory usage: 2.0+ MB


In [854]:
# Verificar la cantidad de nulos para estandarizar (tipo de variable vs tipo de dato)
count_column_types(df_games)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'float'>,Nulos,Nulos %
0,genres,28852.0,3283.0,3283,10.22
1,app_name,32133.0,2.0,2,0.01
2,release_date,30068.0,2067.0,2067,6.43
3,tags,31972.0,163.0,163,0.51
4,specs,31465.0,670.0,670,2.08
5,price,30758.0,1377.0,1377,4.29
6,id,NaN,32135.0,2,0.01
7,developer,28836.0,3299.0,3299,10.27


In [855]:
df_games.head(5)

,genres,app_name,release_date,tags,specs,price,id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,761140.0,Kotoshiro
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",Free To Play,643980.0,Secret Level SRL
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...","['Single-player', 'Multi-player', 'Online Mult...",Free to Play,670290.0,Poolians.com
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",['Single-player'],0.99,767400.0,彼岸领域
4,NaN,Log Challenge,NaN,"['Action', 'Indie', 'Casual', 'Sports']","['Single-player', 'Full controller support', '...",2.99,773570.0,NaN


#### **2- df_games.id**
**Observaciones y conclusiones:**

- Se eliminan los registros duplicados y nulos dado que son inferiores al 1%
- Se renombra la columna a "item_id" para que coincida con el campo relacionado al mismo dato en los otros datasets 
- Se convierte la columna tipo "str" puesto que se considera una variable de tipo categórico

In [856]:
# Encontrar los registros duplicados en la columna "id"
df_games[df_games.duplicated(subset=['id'], keep=False)]

,genres,app_name,release_date,tags,specs,price,id,developer
74,NaN,NaN,NaN,NaN,NaN,19.99,NaN,NaN
13894,['Action'],Wolfenstein II: The New Colossus,2017-10-26,"['Action', 'FPS', 'Gore', 'Violent', 'Alternat...","['Single-player', 'Steam Achievements', 'Full ...",59.99,612880.0,Machine Games
14573,['Action'],Wolfenstein II: The New Colossus,2017-10-26,"['Action', 'FPS', 'Gore', 'Violent', 'Alternat...","['Single-player', 'Steam Achievements', 'Full ...",59.99,612880.0,Machine Games
30961,"['Action', 'Adventure']",Batman: Arkham City - Game of the Year Edition,2012-09-07,"['Action', 'Open World', 'Batman', 'Adventure'...","['Single-player', 'Steam Achievements', 'Steam...",19.99,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [857]:
# Lista de índices a eliminar
indices_a_eliminar = [74, 14573]

# Eliminar los registros de los índices especificados
df_games.drop(indices_a_eliminar, inplace=True)

In [858]:
# Verificar los valores nulos de la columna id
df_games[df_games["id"].isnull()]

,genres,app_name,release_date,tags,specs,price,id,developer
30961,"['Action', 'Adventure']",Batman: Arkham City - Game of the Year Edition,2012-09-07,"['Action', 'Open World', 'Batman', 'Adventure'...","['Single-player', 'Steam Achievements', 'Steam...",19.99,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [859]:
# Eliminar los valores nulos de la columna id
df_games = df_games.dropna(subset=['id'])

# Como todos los id contienen el mismo tipo de dato (float) se le quita la parte decimal y se convierte a str
df_games["id"] = df_games["id"].astype(int).astype(str)

In [860]:
# Renombrar columna "id"
df_games.rename(columns={'id': 'item_id'}, inplace=True)

In [861]:
# Verificar sintaxis de la columna
item_id = df_games["item_id"].unique()
item_id = sorted(item_id)
item_id

['10',
 '1002',
 '100400',
 '10090',
 '100980',
 '10100',
 '10110',
 '10120',
 '10130',
 '10140',
 '10150',
 '10180',
 '10195',
 '10220',
 '102200',
 '10230',
 '10240',
 '102400',
 '10250',
 '102500',
 '102510',
 '102511',
 '10260',
 '102600',
 '102622',
 '102623',
 '10270',
 '102700',
 '102810',
 '102840',
 '102850',
 '104000',
 '104020',
 '104200',
 '10460',
 '104600',
 '104700',
 '104900',
 '10500',
 '105000',
 '105100',
 '10520',
 '105300',
 '105420',
 '105450',
 '105600',
 '105700',
 '105800',
 '105802',
 '10600',
 '10604',
 '10606',
 '10607',
 '10608',
 '10680',
 '10695',
 '10697',
 '107100',
 '107104',
 '107200',
 '107300',
 '107310',
 '107410',
 '107800',
 '107821',
 '108110',
 '108200',
 '108210',
 '108230',
 '108231',
 '108232',
 '108233',
 '108500',
 '108600',
 '108700',
 '108710',
 '108726',
 '108800',
 '109200',
 '109500',
 '109600',
 '109700',
 '109702',
 '11020',
 '11040',
 '110400',
 '11050',
 '110500',
 '110600',
 '110610',
 '110630',
 '110800',
 '110810',
 '111000',
 

#### **2- df_games.release_date**

**Observaciones y conclusiones:**

- Extraer el año en una nueva columna (release_year)
- Se convierte a numérico porque se considera una variable de tipo nominal
- Se elimina esta columna porque los demás datos son irrelevantes
- Se mantienen los nulos

In [862]:
# Extraer el año de "release_date" en "release_year" y borrar "release_date"

# Utilizar expresión regular para extraer números con 4 cifras o más
df_games['release_year'] = df_games['release_date'].str.extract(r'(\d{4,})')

# Convertir la columna a tipo numérico
df_games['release_year'] = pd.to_numeric(df_games['release_year'])

# Eliminar la columna "release_date"
df_games = df_games.drop(columns=["release_date"])


#### **2- df_games.price**

**Observaciones y conclusiones:**

- Extraer los valores numéricos de la columna y reemplazar por 0.0 los que tengan texto pues su mayoría hacen referencia a valores de tipo Free
- Se mantiene en tipo "float" porque se considera de tipo nominal
- Se mantienen los nulos

In [863]:
# Verificar los valores unicos de la columna price
valores_price = sorted(df_games['price'].dropna().unique(), reverse=True)
valores_price

['Third-party',
 'Starting at $499.00',
 'Starting at $449.00',
 'Play the Demo',
 'Play for Free!',
 'Play WARMACHINE: Tactics Demo',
 'Play Now',
 'Install Theme',
 'Install Now',
 'Free to Use',
 'Free to Try',
 'Free to Play',
 'Free To Play',
 'Free Movie',
 'Free Mod',
 'Free HITMAN™ Holiday Pack',
 'Free Demo',
 'Free',
 '995.0',
 '99.99',
 '99.0',
 '9.99',
 '9.98',
 '9.95',
 '9.69',
 '9.0',
 '89.99',
 '87.94',
 '8.99',
 '8.98',
 '79.99',
 '771.71',
 '74.99',
 '74.76',
 '71.7',
 '7.99',
 '7.49',
 '7.0',
 '69.99',
 '64.99',
 '61.99',
 '6.99',
 '6.66',
 '6.49',
 '6.48',
 '6.0',
 '599.0',
 '59.99',
 '59.95',
 '54.99',
 '5.99',
 '5.65',
 '5.49',
 '5.0',
 '499.99',
 '49.99',
 '49.0',
 '44.99',
 '44.98',
 '42.99',
 '41.99',
 '40.0',
 '4.99',
 '4.68',
 '4.49',
 '4.29',
 '4.0',
 '399.99',
 '399.0',
 '39.99',
 '38.85',
 '36.99',
 '34.99',
 '320.0',
 '32.99',
 '31.99',
 '30.0',
 '3.99',
 '3.49',
 '3.39',
 '3.33',
 '3.0',
 '299.99',
 '29.99',
 '29.96',
 '289.99',
 '27.99',
 '27.49',
 '26.9

In [864]:
# Extraer los precios de la columna "price" y reemplazar por 0 los valores que solo contengan str ya que hacen referencia a versiones gratuitas. Los nulos se quedan como nulos

def extraer_numeros_decimales(texto):
    '''
    Extraer los números decimales de una cadena de texto
    '''
    if texto == "nan":
        return texto
    else:
        numeros_decimales = re.findall(r'\d+\.\d+', texto)
        numeros_enteros = re.findall(r'\d', texto)
        if numeros_decimales:
            return float(numeros_decimales[0])
        elif numeros_enteros:
            return float(numeros_enteros[0])
        else:
            return 0.0

# Convertir columna "price" a str para evaluarla
df_games['price'] = df_games['price'].astype(str)

# Aplicar la función a la columna "price"
df_games['price'] = df_games['price'].apply(extraer_numeros_decimales)

# Restaurar valores "nan" en valores de tipo nulos
df_games['price'] = df_games['price'].replace('nan', np.nan)

#### **2- df_games - info**

In [865]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28850 non-null  object 
 1   app_name      32131 non-null  object 
 2   tags          31970 non-null  object 
 3   specs         31463 non-null  object 
 4   price         30755 non-null  float64
 5   item_id       32132 non-null  object 
 6   developer     28834 non-null  object 
 7   release_year  29965 non-null  float64
dtypes: float64(2), object(6)
memory usage: 2.2+ MB


In [866]:
count_column_types(df_games)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'float'>,Nulos,Nulos %
0,genres,28850.0,3282.0,3282,10.21
1,app_name,32131.0,1.0,1,0.00
2,tags,31970.0,162.0,162,0.50
3,specs,31463.0,669.0,669,2.08
4,price,NaN,32132.0,1377,4.29
5,item_id,32132.0,NaN,0,0.00
6,developer,28834.0,3298.0,3298,10.26
7,release_year,NaN,32132.0,2167,6.74


#### **2- df_games - checkpoint**

In [867]:
# Creación punto de restauración 
df_games_2 = df_games.copy()
df_games_2 = df_games_2.reset_index(drop=True)    # Restablecer el índice

In [868]:
# Carga punto de restauración 
df_games = df_games_2.copy()

### **2- df_reviews**

- (Duplicados, nulos, formato columnas)<br>

In [869]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   posted     59305 non-null  object
 2   item_id    59305 non-null  int64 
 3   recommend  59305 non-null  bool  
 4   review     59275 non-null  object
dtypes: bool(1), int64(1), object(3)
memory usage: 1.9+ MB


In [870]:
count_column_types(df_reviews)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,<class 'bool'>,<class 'float'>,Nulos,Nulos %
0,user_id,59305.0,NaN,NaN,NaN,0,0.00
1,posted,59305.0,NaN,NaN,NaN,0,0.00
2,item_id,NaN,59305.0,NaN,NaN,0,0.00
3,recommend,NaN,NaN,59305.0,NaN,0,0.00
4,review,59275.0,NaN,NaN,30.0,30,0.05


In [871]:
df_reviews.head(5)

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


#### **2- df_reviews - duplicados**

**Observaciones y conclusiones:**

- Dado que hay dos columnas que contienen registros unicos (user_id, item_id) es preciso sacar una llave principal anidada entre las dos parar descartar valores duplicados

In [872]:
# Verificar sintaxis de la columna
item_id = df_reviews["item_id"].unique()
item_id = sorted(item_id)
item_id

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 130,
 220,
 240,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 410,
 420,
 440,
 500,
 550,
 570,
 620,
 630,
 730,
 1200,
 1250,
 1280,
 1510,
 1520,
 1670,
 1690,
 1840,
 1900,
 1930,
 2100,
 2200,
 2270,
 2280,
 2290,
 2300,
 2310,
 2320,
 2400,
 2420,
 2450,
 2500,
 2570,
 2590,
 2600,
 2610,
 2620,
 2630,
 2640,
 2700,
 2760,
 2810,
 2820,
 2850,
 2870,
 2990,
 3050,
 3170,
 3260,
 3320,
 3370,
 3390,
 3450,
 3470,
 3480,
 3483,
 3590,
 3620,
 3700,
 3710,
 3720,
 3730,
 3800,
 3830,
 3900,
 3910,
 3920,
 4000,
 4500,
 4540,
 4550,
 4560,
 4570,
 4580,
 4700,
 4720,
 4760,
 4780,
 4830,
 4850,
 4920,
 6000,
 6020,
 6030,
 6060,
 6100,
 6120,
 6210,
 6220,
 6310,
 6370,
 6420,
 6550,
 6800,
 6830,
 6850,
 6860,
 6870,
 6880,
 6900,
 6910,
 6950,
 6980,
 7000,
 7020,
 7200,
 7510,
 7520,
 7600,
 7660,
 7670,
 7730,
 7760,
 7770,
 7830,
 7840,
 7860,
 7940,
 8000,
 8080,
 8140,
 8170,
 8190,
 8230,
 8310,
 8340,
 8400,
 8500,
 8600,
 8800,
 8850,

In [873]:
# Verificar sintaxis de la columna
item_id = df_reviews["user_id"].unique()
item_id = sorted(item_id)
item_id

['--000--',
 '--ace--',
 '--ionex--',
 '-2SV-vuLB-Kg',
 '-Azsael-',
 '-Beave-',
 '-GM-Dragon',
 '-I_AM_EPIC-',
 '-Kenny',
 '-Mad-',
 '-PRoSlayeR-',
 '-SEVEN-',
 '-SatansLittleHelper-',
 '-Thyme-',
 '-Ultrix',
 '-Zovix-',
 '-_PussyDestroyer_-',
 '-kainey9777',
 '0-3-0',
 '00000000000000000001227',
 '001000111000111000010',
 '00284702',
 '00454211432342',
 '00690069006900',
 '00810cccyyykkk',
 '0099654321891111',
 '00True',
 '01001000-01101001',
 '011011010111001000101110',
 '011111135489484797',
 '01189958889189157253',
 '01221733',
 '0132489',
 '0136849195',
 '01shan',
 '022899',
 '03092002',
 '04061993',
 '04092013',
 '0412734634',
 '042153100',
 '04280122',
 '0432-026-no',
 '0445233',
 '0468313256',
 '05041129',
 '05461025',
 '0700464574757',
 '07824',
 '07846813956825',
 '08254669696969696969',
 '0854029092',
 '0896398616',
 '0903512455',
 '09106',
 '091263',
 '094553552',
 '095732',
 '095843067',
 '09876543231',
 '098766catdogbnmnnn',
 '09879655452567',
 '10000000thVisitor',
 '1001

In [874]:
# Revisar duplicados creando llave anidada: user_id + item_id
df_reviews["llave"] = df_reviews["user_id"].astype(str) + df_reviews["item_id"].astype(str)

# Verificar duplicados en la columna "llave"
duplicados = df_reviews['llave'].duplicated(keep='first')
duplicados.sum() / 2

437.0

In [875]:
# Eliminar registros donde la columna "llave" esté duplicada
df_reviews = df_reviews[~duplicados]

#### **2- df_reviews.item_id**

**Observaciones y conclusiones:**

- Se convierte a tipo "str" porque se considera una variable de tipo categórica

In [876]:
# Convertir la columna item_id a tipo str
df_reviews["item_id"] = df_reviews["item_id"].astype(str)

#### **2- df_reviews.posted**

**Observaciones y conclusiones:**

- Se evidencian años faltantes así que se extraen los años en una columna y se revisa el minimo y el maximo para encontrar el rango. Dentro de este análisis se encuentra la fecha "ferbero 29" sin año lo cual indica q su año debe ser un consecutivo proximo al rango de años tratados. Los años bisiestos más próximos son 2008, 2012 y 2016. Se descarta 2008 por ser el valor más lejano al rango y se elije el año 2016 para inputar los años faltantes pues encaja como un valor extremo en el rango
- Se convierte a tipo "str" en formato YYYY-MM-DD
- Se ajusta el nombre del campo a "posted_date" para que sea más descriptivo

In [877]:
# Verificar los valores unicos de la columna posted
valores_posted = sorted(df_reviews['posted'].dropna().unique(), reverse=True)
valores_posted

['Posted September 9.',
 'Posted September 9, 2015.',
 'Posted September 9, 2014.',
 'Posted September 9, 2013.',
 'Posted September 9, 2012.',
 'Posted September 9, 2011.',
 'Posted September 8.',
 'Posted September 8, 2015.',
 'Posted September 8, 2014.',
 'Posted September 8, 2013.',
 'Posted September 8, 2012.',
 'Posted September 8, 2011.',
 'Posted September 7.',
 'Posted September 7, 2015.',
 'Posted September 7, 2014.',
 'Posted September 7, 2013.',
 'Posted September 7, 2012.',
 'Posted September 7, 2011.',
 'Posted September 6.',
 'Posted September 6, 2015.',
 'Posted September 6, 2014.',
 'Posted September 6, 2013.',
 'Posted September 6, 2012.',
 'Posted September 6, 2011.',
 'Posted September 5.',
 'Posted September 5, 2015.',
 'Posted September 5, 2014.',
 'Posted September 5, 2013.',
 'Posted September 5, 2012.',
 'Posted September 4.',
 'Posted September 4, 2015.',
 'Posted September 4, 2014.',
 'Posted September 4, 2013.',
 'Posted September 4, 2012.',
 'Posted Septemb

In [878]:
# Ajustar tipo de dato columna "posted" de str a tipo DateTime

# Convertir la columna 'posted' a minúsculas
df_reviews['posted'] = df_reviews['posted'].str.lower()

# Eliminar caracteres innecesarios
df_reviews['posted'] = df_reviews['posted'].str.replace('posted ', '')
df_reviews['posted'] = df_reviews['posted'].str.replace(',', '')
df_reviews['posted'] = df_reviews['posted'].str.replace('.', '')
df_reviews['posted'] = df_reviews['posted'].str.replace(' ', '-')

In [879]:
# Extraer el año de "posted" en "posted_year"

# Utilizar expresión regular para extraer números con 4 cifras o más
df_reviews['posted_year'] = df_reviews['posted'].str.extract(r'(\d{4,})')

# Convertir la columna a tipo numérico
df_reviews['posted_year'] = pd.to_numeric(df_reviews['posted_year'])

# Sacar el valor minimo y máximo para la columna "posted_year"
print(df_reviews['posted_year'].min())
print(df_reviews['posted_year'].max())

2010.0
2015.0


In [880]:
# Revisar las fechas que tienen años nulos para encontrar un patrón
valores = sorted(df_reviews["posted"][df_reviews["posted_year"].isnull()].unique())
valores

['april-1',
 'april-10',
 'april-11',
 'april-12',
 'april-13',
 'april-14',
 'april-15',
 'april-16',
 'april-17',
 'april-18',
 'april-19',
 'april-2',
 'april-20',
 'april-21',
 'april-22',
 'april-23',
 'april-24',
 'april-25',
 'april-26',
 'april-27',
 'april-28',
 'april-29',
 'april-3',
 'april-30',
 'april-4',
 'april-5',
 'april-6',
 'april-7',
 'april-8',
 'april-9',
 'august-1',
 'august-10',
 'august-11',
 'august-12',
 'august-13',
 'august-14',
 'august-15',
 'august-16',
 'august-17',
 'august-18',
 'august-19',
 'august-2',
 'august-20',
 'august-21',
 'august-22',
 'august-23',
 'august-24',
 'august-25',
 'august-26',
 'august-27',
 'august-28',
 'august-29',
 'august-3',
 'august-30',
 'august-31',
 'august-4',
 'august-5',
 'august-6',
 'august-7',
 'august-8',
 'august-9',
 'february-1',
 'february-10',
 'february-11',
 'february-12',
 'february-13',
 'february-14',
 'february-15',
 'february-16',
 'february-17',
 'february-18',
 'february-19',
 'february-2',
 'fe

In [881]:
def convertir_fecha(texto):
    '''
    Convertir una fecha con mes en palabras y en formato MM-DD-YYYY a un formato YYYY-MM-DD donde año, mes y dia son numericos
    '''
    try:
        # Intentar analizar la fecha con el formato "%B-%d-%Y"
        fecha = datetime.strptime(texto, "%B-%d-%Y")
        return fecha.strftime("%Y-%m-%d")
    except ValueError:
        try:
            # Si no coincide, intentar con el formato "%B-%d"
            fecha = datetime.strptime(texto, "%B-%d")
            return fecha.strftime("2016-%m-%d")
        except ValueError:
            # Si no se puede analizar o es una fecha inválida, devolver 'NaN'
            return None

# Aplicar la función a la columna 'posted'
df_reviews['posted_date'] = df_reviews['posted'].apply(convertir_fecha)

# Revisar si hay fechas que quedaron nulos porque no se pudieron editar en formato fecha
df_reviews["posted"][df_reviews["posted_date"].isnull()].unique()

array(['february-29'], dtype=object)

In [882]:
# Reemplazar el valor nulo en la fecha "february-29" por "2016-02-29" 
df_reviews['posted_date'].fillna("2016-02-29", inplace=True)

# Eliminar columnas que no se van a usar
columnas = ["posted_year", "posted", "llave"]
df_reviews.drop(columns=columnas, inplace=True)

# Verificar estado del df
count_column_types(df_reviews)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'bool'>,<class 'float'>,Nulos,Nulos %
0,user_id,58431.0,NaN,NaN,0,0.00
1,item_id,58431.0,NaN,NaN,0,0.00
2,recommend,NaN,58431.0,NaN,0,0.00
3,review,58401.0,NaN,30.0,30,0.05
4,posted_date,58431.0,NaN,NaN,0,0.00


#### **2- df_reviews.review**

**Observaciones y conclusiones:**

- Se crea la columna 'sentiment_analysis' (int) aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral (o si no hay reseña) y '2' si es positivo.
- Se crea la columna "sentiment_label" de tipo "str"
- Se elimina la columna "review"

In [883]:
# Eliminar espacios nulos para evaluar toda la columna como str
df_reviews["review"].fillna("", inplace=True)

# Convertir todos los elementos de la columna a cadena
df_reviews["review"] = df_reviews["review"].astype(str)

# Ver los tipos de datos que manejan las reseñas
resenas = df_reviews["review"].unique()
resenas = sorted(resenas)
resenas

['',
 "\tGET IT IT' SO GOOOD",
 '\tim on a mac and i cant load up the game it keeps on dissapering or something',
 ' ',
 '                                                                                     *EVERQUEST**Hundreds of thousands of in game items, with them being added continueously!*Hundred of zones*Great Crafting system*Wonderful Community*Nice Graphics*Never ending game!Hits everypoint on a great MMO abosultely superb!! And totally addictive!Love this game 10/10',
 '                                                 DOTA 2  BE WARNED there is no cure for this game11/10',
 '             "My life is complete       Nothing else could compare        Receiving number oneReaching new heights in technology         Realizing one\'s dream         Is not worth to beauty of this game on my monitor screen."',
 '       ËøôÊ¨æÊ∏∏ÊàèÁöÑÁâπËâ≤Â∞±ÊòØÂú∞ÂõæÂæàÂ§ßÔºåÁúüÁöÑÈùûÂ∏∏Â§ßÔºå‰ΩÜÊòØÊ∏∏ÊàèÁ≥ªÁªüÁöÑÂ§öÊ†∑ÊÄßÂæàÁº∫‰πèÔºå‰ª•Ëá≥‰∫éÈô§‰∫ÜÁé©‰ªªÂä°‰πãÂ§ñÊàëÊ≤°Êúâ‰ªª‰ΩïÁêÜÁî±ÂéªÁé©Ëøô‰∏™Ê∏∏Êà

In [884]:
# Verificar que las celdas tengan cadenas con caracteres alfabéticos  para para evaluar

def contains_words_after_cleaning(text):
    '''
    Función para verificar si una cadena contiene palabras después de eliminar caracteres no alfabéticos
    '''

    # Utiliza una expresión regular para eliminar caracteres no alfabéticos y convertir todo a minúsculas
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    
    # Divide la cadena en palabras
    words = cleaned_text.split()
    
    # Verifica si hay al menos una palabra en la cadena
    return len(words) > 0

# Aplica la función contains_words_after_cleaning a la columna df_reviews["review"]
df_reviews["valid_review"] = df_reviews["review"].apply(contains_words_after_cleaning)

# Dejar solo los registros con caractéres alfabeticos y reemplazar con "" los registros nulos 
df_reviews["review_2"] = df_reviews["review"][df_reviews["valid_review"] == True]
df_reviews["review_2"].fillna("", inplace=True)

In [885]:
# Examinar y validar los valores resultantes y la cantidad por cada uno
df_reviews["valid_review"].value_counts()

valid_review
True     56925
False     1506
Name: count, dtype: int64

In [886]:
# Revisar las celdas que contienen reviews "validas" que se tomarán en cuenta
words_review = df_reviews["review"][df_reviews["valid_review"] == True].unique()
words_review = sorted(words_review)
words_review

["\tGET IT IT' SO GOOOD",
 '\tim on a mac and i cant load up the game it keeps on dissapering or something',
 '                                                                                     *EVERQUEST**Hundreds of thousands of in game items, with them being added continueously!*Hundred of zones*Great Crafting system*Wonderful Community*Nice Graphics*Never ending game!Hits everypoint on a great MMO abosultely superb!! And totally addictive!Love this game 10/10',
 '                                                 DOTA 2  BE WARNED there is no cure for this game11/10',
 '             "My life is complete       Nothing else could compare        Receiving number oneReaching new heights in technology         Realizing one\'s dream         Is not worth to beauty of this game on my monitor screen."',
 '       ËøôÊ¨æÊ∏∏ÊàèÁöÑÁâπËâ≤Â∞±ÊòØÂú∞ÂõæÂæàÂ§ßÔºåÁúüÁöÑÈùûÂ∏∏Â§ßÔºå‰ΩÜÊòØÊ∏∏ÊàèÁ≥ªÁªüÁöÑÂ§öÊ†∑ÊÄßÂæàÁº∫‰πèÔºå‰ª•Ëá≥‰∫éÈô§‰∫ÜÁé©‰ªªÂä°‰πãÂ§ñÊàëÊ≤°Êúâ‰ªª‰ΩïÁêÜÁî±ÂéªÁé©Ëøô‰∏™Ê∏∏ÊàèÔºåÊ∏∏ÊàèÂ

In [887]:
# Revisar las celdas que contienen reviews "no validas" que no se tomarán en cuenta
words_review = df_reviews["review"][df_reviews["valid_review"] == False].unique()
words_review = sorted(words_review)
words_review

['',
 ' ',
 '    ,',
 '    .',
 ' ÏïÑÏ£º Ï¢ãÏùÄ Í≤åÏûÑ. Î™ÖÏûë Î™ÖÏûë',
 ' ‡∏†‡∏≤‡∏Ñ‡∏ô‡∏µ‡πâ‡∏°‡∏±‡∏ô‡∏™‡πå‡∏°‡∏≤‡∏Å‡∏≠‡∏¥‡∏≠‡∏¥ ‡πÄ‡∏ô‡∏∑‡πà‡∏≠‡πÄ‡∏£‡∏∑‡πà‡∏≠‡∏á‡πÉ‡∏´‡∏°‡πà ‡πÄ‡∏Å‡∏µ‡πà‡∏¢‡∏ß‡∏Å‡∏±‡∏ö‡πÇ‡∏à‡∏£‡∏™‡∏•‡∏±‡∏î‡∏ñ‡∏∑‡∏≠‡∏ß‡πà‡∏≤‡∏™‡∏ô‡∏∏‡∏Å‡πÄ‡∏•‡∏¢‡∏Ñ‡∏£‡∏±‡∏ö ‡∏Å‡∏£‡∏≤‡∏ü‡∏ü‡∏¥‡∏Ñ ‡∏Å‡πá‡∏Ñ‡∏•‡πâ‡∏≤‡∏¢‡∏Ñ‡∏•‡∏∂‡∏á‡∏Å‡∏±‡∏ö‡∏†‡∏≤‡∏Ñ 3  ‡πÅ‡∏ï‡πà‡∏Ñ‡∏ß‡∏≤‡∏°‡∏°‡∏±‡∏ô‡∏™‡πå‡∏¢‡∏±‡∏á‡∏Ñ‡∏á‡∏≠‡∏¢‡∏π‡πà',
 ' ‰∏ÄÂÆö‰∏çË¶ÅÂú®Ê∏∏ÊàèÊó∂Èó¥Âà∞200Â∞èÊó∂‰ª•ÂâçÁªôÂ•ΩËØÑÔºåÂõ†‰∏∫ÂæàÈáçË¶ÅÊâÄ‰ª•ÊàëÊáíÂæóËØ¥Á¨¨‰∫åÈÅçÔºÅÔºÅÔºÅ',
 '!',
 '!!',
 '!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '" Í∑∏ÎûòÌîΩÎèÑ ÎÇòÏÅòÏßÄ ÏïäÍ≥†.. Ï†ÅÎãπÌûà ÎÇÆÏùÄ Ïö©ÎüâÏóê.. Ï°∞Ïù¥Ìå®ÎìúÎ°úÎèÑ ÌïòÏãúÎ©¥ Îçî Ïû¨ÎØ∏Î•º ÎäêÎÅºÏã§ Ïàò ÏûàÏúºÏãúÎãà Î¨¥Î£åÎãàÍπå ÌïúÎ≤àÏùÄ Îã§Ïö¥Î∞õÏïÑÏÑú Ìï¥Î≥¥ÏÑ∏Ïöî ~!!',
 '" Í≥µÏ§ëÏóêÏÑú ÏµúÍ≥†Ïùò Ïï°ÏÖòÏùÑ ÏÑ†Î≥¥Ïù¥Îäî Î∞îÏù¥Ïò§ÏáºÌÅ¨ Ïù∏ÌîºÎãàÌã∞Ïù∏Îç∞Ïöî ! Íº≠ ÌïúÎ≤àÏùÄ ÏßÄÎ•¥Í≥† Ìï¥Î≥¥ÏãúÎäî Í±∏ Í∞ïÏ∂îÎìúÎ¶ΩÎãàÎã§ !~ Ï†ïÎßêÎ°ú ÌõÑÌöåÏóÜÎäî ÏãúÍ∞ÑÏùÑ Î≥¥ÎÇ¥Ïã§ Ïàò ÏûàÏ

In [888]:
# Descargar los datos necesarios para el análisis de sentimiento
nltk.download('vader_lexicon') 

# Inicializar SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to C:\Users\Ivan
[nltk_data]     Rojas\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [889]:
# Calcular el score para cada review y añadir la columna al dataframe
df_reviews["review_score"] = df_reviews["review"].apply(lambda x: sia.polarity_scores(x)["compound"])

# Calcular la columna sent_score según los criterios
df_reviews["sentiment_analysis"] = df_reviews["review_score"].apply(lambda score: 2 if score > 0 else 1 if score == 0 else 0)

# Asignar los labels de sentiment de acuerdo al score
df_reviews["sentiment_label"] = df_reviews["sentiment_analysis"].apply(lambda score: "positive" if score > 1 else "negative" if score < 1 else "neutral")    

In [890]:
# Revisar sómo se tomaro los valores ""
df_reviews[["review", "review_score", "sentiment_analysis", "sentiment_label"]][df_reviews["review"] == ""].head(5)

,review,review_score,sentiment_analysis,sentiment_label
3095,,0.0,1,neutral
4616,,0.0,1,neutral
15975,,0.0,1,neutral
20478,,0.0,1,neutral
22049,,0.0,1,neutral


In [891]:
# Comprobar los valores numericos asignados
df_reviews["sentiment_analysis"].unique()

array([2, 1, 0], dtype=int64)

In [892]:
# Comprobar las etiquetas asignadas
df_reviews["sentiment_label"].unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [893]:
df_reviews.head(3)

,user_id,item_id,recommend,review,posted_date,valid_review,review_2,review_score,sentiment_analysis,sentiment_label
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,True,Simple yet with great replayability. In my opi...,0.8481,2,positive
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15,True,It's unique and worth a playthrough.,0.2263,2,positive
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,True,Great atmosphere. The gunplay can be a bit chu...,0.9117,2,positive


#### **2- df_review - info**

**Observaciones y conclusiones:**

- Se eliminan las columnas resultantes de analisis temporales:
    - review
    - valid_review
    - review_2
    - review_score

In [894]:
# Eliminar columnas irrelevantes y resetear el indice
columnas_a_eliminar = ["review", "valid_review", "review_2", "review_score"]
df_reviews.drop(columnas_a_eliminar, axis=1, inplace=True)
df_reviews.reset_index(drop=True, inplace=True)

In [895]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58431 non-null  object
 1   item_id             58431 non-null  object
 2   recommend           58431 non-null  bool  
 3   posted_date         58431 non-null  object
 4   sentiment_analysis  58431 non-null  int64 
 5   sentiment_label     58431 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 2.3+ MB


In [896]:
count_column_types(df_reviews)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'bool'>,<class 'int'>,Nulos,Nulos %
0,user_id,58431.0,NaN,NaN,0,0.0
1,item_id,58431.0,NaN,NaN,0,0.0
2,recommend,NaN,58431.0,NaN,0,0.0
3,posted_date,58431.0,NaN,NaN,0,0.0
4,sentiment_analysis,NaN,NaN,58431.0,0,0.0
5,sentiment_label,58431.0,NaN,NaN,0,0.0


#### **2- df_review - checkpoint**

In [897]:
# Creación punto de restauración 
df_reviews_2 = df_reviews.copy()
df_reviews_2 = df_reviews_2.reset_index(drop=True)    # Restablecer el índice

In [898]:
# Carga pto restauración
df_reviews = df_reviews_2.copy()

### **2- df_items**

- (Duplicados, nulos, formato columnas)<br>

In [899]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   user_url          object
 3   item_id           int64 
 4   playtime_forever  int64 
dtypes: int64(3), object(2)
memory usage: 196.6+ MB


In [900]:
count_column_types(df_items)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,Nulos,Nulos %
0,user_id,5153209.0,NaN,0,0.0
1,items_count,NaN,5153209.0,0,0.0
2,user_url,5153209.0,NaN,0,0.0
3,item_id,NaN,5153209.0,0,0.0
4,playtime_forever,NaN,5153209.0,0,0.0


In [901]:
df_items.head(5)

,user_id,items_count,user_url,item_id,playtime_forever
0,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,10,6
1,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,20,0
2,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,30,7
3,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,40,0
4,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,50,0


#### **2- df_items - duplicados**

**Observaciones y conclusiones:**

- Dado que hay dos columnas que contienen registros unicos (user_id, item_id) es preciso sacar una llave principal anidada entre las dos parar descartar valores duplicados

In [902]:
# Verificar sintaxis de la columna
item_id = df_items["item_id"].unique()
item_id = sorted(item_id)
item_id

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 100,
 130,
 220,
 240,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 500,
 550,
 620,
 730,
 1002,
 1200,
 1210,
 1230,
 1250,
 1280,
 1300,
 1309,
 1313,
 1320,
 1500,
 1510,
 1520,
 1530,
 1600,
 1610,
 1630,
 1640,
 1670,
 1690,
 1700,
 1900,
 1920,
 1930,
 2100,
 2130,
 2200,
 2210,
 2270,
 2280,
 2290,
 2300,
 2310,
 2320,
 2330,
 2340,
 2350,
 2360,
 2370,
 2390,
 2400,
 2420,
 2430,
 2450,
 2500,
 2520,
 2525,
 2540,
 2545,
 2550,
 2570,
 2590,
 2600,
 2610,
 2620,
 2630,
 2640,
 2680,
 2690,
 2700,
 2710,
 2720,
 2760,
 2780,
 2790,
 2800,
 2810,
 2820,
 2840,
 2850,
 2870,
 2900,
 2910,
 2920,
 2930,
 2990,
 3000,
 3010,
 3020,
 3050,
 3130,
 3150,
 3160,
 3170,
 3200,
 3220,
 3230,
 3260,
 3270,
 3300,
 3310,
 3320,
 3330,
 3340,
 3350,
 3360,
 3370,
 3380,
 3390,
 3400,
 3410,
 3420,
 3430,
 3440,
 3450,
 3460,
 3470,
 3480,
 3483,
 3490,
 3500,
 3510,
 3520,
 3530,
 3540,
 3560,
 3570,
 3580,
 3590,
 3600,
 3610,
 3620,
 3630,
 36

In [903]:
# Verificar sintaxis de la columna
item_id = df_items["user_id"].unique()
item_id = sorted(item_id)
item_id

['--000--',
 '--ace--',
 '--ionex--',
 '-2SV-vuLB-Kg',
 '-404PageNotFound-',
 '-AnimeIsMyThing-',
 '-Azsael-',
 '-Beave-',
 '-Encore-',
 '-GM-Dragon',
 '-I_AM_EPIC-',
 '-JT',
 '-Kenny',
 '-KillZone-',
 '-Mad-',
 '-Naughty-',
 '-PRoSlayeR-',
 '-SEVEN-',
 '-SatansLittleHelper-',
 '-Thyme-',
 '-Ultrix',
 '-Uplink-',
 '-Zovix-',
 '-_PussyDestroyer_-',
 '-_skIzZ_-',
 '-fastrvrs-',
 '-iBubble',
 '-kainey9777',
 '-xKyox-',
 '0-3-0',
 '00000000000000000001227',
 '00000003010',
 '0000FF',
 '000Infinity000',
 '001000111000111000010',
 '001002130882',
 '001010001',
 '0041287',
 '00454211432342',
 '005577',
 '00690069006900',
 '0071945',
 '007Pro',
 '007james_bond',
 '007thesilence',
 '00810cccyyykkk',
 '008port',
 '0099654321891111',
 '00True',
 '00nana',
 '01001000-01101001',
 '010195345',
 '0102705195',
 '010607',
 '011000100001000111',
 '011011010111001000101110',
 '011111135489484797',
 '01120477677',
 '0112937783',
 '01189958889189157253',
 '0118999881999119',
 '01221733',
 '0136849195',
 '0

In [904]:
# Revisar duplicados creando llave anidada: user_id + item_id
df_items["llave"] = df_items["user_id"].astype(str) + df_items["item_id"].astype(str)

# Verificar duplicados en la columna "llave"
duplicados = df_items['llave'].duplicated(keep='first')
duplicados.sum() / 2

29563.5

In [905]:
# Eliminar registros donde la columna "llave" esté duplicada
df_items = df_items[~duplicados]

#### **2- df_items.item_id**
**Observaciones y conclusiones:**

- Se convierte a tipo "str" porque se considera una variable de tipo categórica

In [906]:
# Convertir la columna item_id a tipo str
df_items["item_id"] = df_items["item_id"].astype(str)

#### **2- df_items.playtime_mins**
**Observaciones y conclusiones:**

- Se ajusta el nombre de la columna para que sea más corto y descriptivo con el contenido

In [907]:
# Renombrar columna
df_items.rename(columns={'playtime_forever': 'playtime_mins'}, inplace=True)

#### **2- df_items - info**

**Observaciones y conclusiones:**

- Se eliminan las columnas resultantes de analisis temporales:
    - llave

In [908]:
# Eliminar columnas irrelevantes y resetear el indice
columnas_a_eliminar = ["llave"]
df_items.drop(columnas_a_eliminar, axis=1, inplace=True)
df_items.reset_index(drop=True, inplace=True)

In [909]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        object
 1   items_count    int64 
 2   user_url       object
 3   item_id        object
 4   playtime_mins  int64 
dtypes: int64(2), object(3)
memory usage: 194.3+ MB


In [910]:
count_column_types(df_items)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,Nulos,Nulos %
0,user_id,5094082.0,NaN,0,0.0
1,items_count,NaN,5094082.0,0,0.0
2,user_url,5094082.0,NaN,0,0.0
3,item_id,5094082.0,NaN,0,0.0
4,playtime_mins,NaN,5094082.0,0,0.0


#### **2- df_items - checkpoint**

In [911]:
# Creación punto de restauración 
df_items_2 = df_items.copy()
df_items_2 = df_items_2.reset_index(drop=True)    # Restablecer el índice

In [912]:
# Carga pto restauración
df_items = df_items_2.copy()

## **3- CREACION DE NUEVOS DATASETS**

### **3- df_userdata**

- df_userdata = def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
    <br>
    - df_items.user_id = (str)
    - df_items.item_id = (str)
    - df_items.items_count = (int)
    - df_games.price = (float)<br>

**Observaciones y conclusiones:**

A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
- user_id (str) = id para cada usuario
- money_spent (float) = dinero gastado por cada usuario
- total_recommend (float) = recomendaciones totales por cada usuario
- items_count (int) = cantidad total de items por cada usuario

In [913]:
# nuevo dataframe
df_1 = df_items[["user_id", "item_id", "items_count"]].copy()
df_2 = df_games[["item_id", "price"]].copy()

# Realizar un left join
df_userdata = df_1.merge(df_2, on='item_id', how='left')

In [914]:
df_userdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      object 
 1   item_id      object 
 2   items_count  int64  
 3   price        float64
dtypes: float64(1), int64(1), object(2)
memory usage: 155.5+ MB


In [915]:
count_column_types(df_userdata)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,<class 'float'>,Nulos,Nulos %
0,user_id,5094082.0,NaN,NaN,0,0.00
1,item_id,5094082.0,NaN,NaN,0,0.00
2,items_count,NaN,5094082.0,NaN,0,0.00
3,price,NaN,NaN,5094082.0,931784,18.29


#### **3- df_userdata_reduced**

In [916]:
# nuevo dataframe reducido
df_userdata_reduced = df_userdata.drop_duplicates(subset='user_id')
df_userdata_reduced = df_userdata_reduced[["user_id", "items_count"]]
df_userdata_reduced

,user_id,items_count
0,76561197970982479,277
277,js41637,888
1165,evcentric,137
1302,Riot-Punch,328
1630,doctr,541
...,...,...
5093551,76561198320136420,321
5093872,ArkPlays7,4
5093876,76561198323066619,22
5093898,76561198326700687,177


#### **3- df_userdata_reduced.money_spent**

In [917]:
# Sumar el conteo de "price" agrupando por "user_id"
df_money_spent = df_userdata.groupby('user_id')['price'].sum().reset_index()
df_money_spent.rename(columns={'price': 'money_spent'}, inplace=True)

# Realizar un left join
df_userdata_reduced = df_userdata_reduced.merge(df_money_spent, on='user_id', how='left')
df_userdata_reduced

,user_id,items_count,money_spent
0,76561197970982479,277,3424.31
1,js41637,888,8553.07
2,evcentric,137,1584.90
3,Riot-Punch,328,3377.22
4,doctr,541,6760.32
...,...,...,...
70907,76561198320136420,321,673.67
70908,ArkPlays7,4,134.97
70909,76561198323066619,22,0.00
70910,76561198326700687,177,375.83


#### **3- df_userdata_reduced.total_recommend**

In [ ]:
# Sumar el conteo de "recommend" agrupando por "user_id"
df_total_recommend = df_reviews.groupby('user_id')['recommend'].sum().reset_index()
df_total_recommend.rename(columns={'recommend': 'total_recommend'}, inplace=True)

# Realizar un left join
df_userdata_reduced = df_userdata_reduced.merge(df_total_recommend, on='user_id', how='left')

# agregar la columna de "porcentaje de recomendacion"
df_userdata_reduced["recommend_percent"] = df_userdata_reduced["total_recommend"] / df_userdata_reduced["items_count"] * 100
df_userdata_reduced["recommend_percent"] = df_userdata_reduced["recommend_percent"].round(2)

df_userdata_reduced

#### **3- df_userdata_reduced - Nulos**

In [919]:
count_column_types(df_userdata_reduced)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,<class 'float'>,Nulos,Nulos %
0,user_id,70912.0,NaN,NaN,0,0.00
1,items_count,NaN,70912.0,NaN,0,0.00
2,money_spent,NaN,NaN,70912.0,0,0.00
3,total_recommend,NaN,NaN,70912.0,48299,68.11


In [920]:
# Cambiar los valores nulos en la columna total_recommend por 0
df_userdata_reduced["total_recommend"] = df_userdata_reduced["total_recommend"].fillna(0)

#### **3- df_userdata_reduced - info**

In [ ]:
# ver valores unicos de price
df_userdata[""]

In [921]:
count_column_types(df_userdata_reduced)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,<class 'float'>,Nulos,Nulos %
0,user_id,70912.0,NaN,NaN,0,0.0
1,items_count,NaN,70912.0,NaN,0,0.0
2,money_spent,NaN,NaN,70912.0,0,0.0
3,total_recommend,NaN,NaN,70912.0,0,0.0


In [956]:
df_userdata_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70912 entries, 0 to 70911
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          70912 non-null  object 
 1   items_count      70912 non-null  int64  
 2   money_spent      70912 non-null  float64
 3   total_recommend  70912 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.2+ MB


### **3- df_countreviews**

- df_countreviews = def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.
    <br>
    - df_reviews.user_id = (str)
    - df_reviews.posted_date = (datetime)
    - df_reviews.recommend = (bool)<br>

**Observaciones y conclusiones:**

- Cada usuario puede tener solo 1 review por item y aunque pueda editar esa review en repetidas ocasiones igualmente será tenida en cuenta como 1 sola.
- Cantidad de usuarios que publicaron reviews entre esas fechas. NO es cantidad de reviews publicadas en esa fecha
- Porcentaje de recomendación de los True respecto a toda la cantidad de publicaciones entre esas fechas

In [970]:
# nuevo dataframe
df_countreviews = df_reviews[["user_id", "posted_date", "recommend"]].copy()

# Usa str.replace para eliminar los guiones
df_countreviews['posted_date_num'] = df_countreviews['posted_date'].str.replace('-', '')

#### **3- df_countreviews - info**

In [971]:
count_column_types(df_countreviews)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'bool'>,Nulos,Nulos %
0,user_id,58431.0,NaN,0,0.0
1,posted_date,58431.0,NaN,0,0.0
2,recommend,NaN,58431.0,0,0.0
3,posted_date_num,58431.0,NaN,0,0.0


In [925]:
df_countreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      58431 non-null  object
 1   posted_date  58431 non-null  object
 2   recommend    58431 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 970.2+ KB


### **3- df_genre**

- df_genre = def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.
    <br>
    - df_items.item_id = (str)
    - df_items.playtime_hours = (int)
    - df_games.genres = (str / list)

**Observaciones y conclusiones:**

A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
- genre (str) = genero unico dentro de la lista total de generos
- ranking (int) = puesto que ocupa cada género de acuerdo a PlayTimeForever

In [926]:
# nuevo dataframe
df_1 = df_items[["item_id", "playtime_mins"]].copy()
df_2 = df_games[["item_id", "genres"]].copy()

# Realizar un left join
df_genre = df_1.merge(df_2, on='item_id', how='left')

df_genre

,item_id,playtime_mins,genres
0,10,6,['Action']
1,20,0,['Action']
2,30,7,['Action']
3,40,0,['Action']
4,50,0,['Action']
...,...,...,...
5094077,346330,0,"['Action', 'Adventure', 'Free to Play', 'Indie..."
5094078,373330,0,NaN
5094079,388490,3,"['Adventure', 'Free to Play']"
5094080,521570,4,"['Casual', 'Free to Play', 'Indie']"


In [927]:
# Eliminar campos irrelevantes
df_genre = df_genre.drop(columns='item_id')

# Eliminar registros donde no hay ningun genero
df_genre.dropna(subset=['genres'], inplace=True)

# Eliminar registros donde "playtime_mins" es igual a 0
df_genre = df_genre[df_genre["playtime_mins"] != 0]

In [928]:
# Sumar el conteo de "playtime_mins" agrupando por "genres"
df_playtime_mins = df_genre.groupby('genres')['playtime_mins'].sum().reset_index()
df_playtime_mins

,genres,playtime_mins
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",37249
1,"['Action', 'Adventure', 'Casual', 'Free to Pla...",37824
2,"['Action', 'Adventure', 'Casual', 'Free to Pla...",366794
3,"['Action', 'Adventure', 'Casual', 'Free to Pla...",50337743
4,"['Action', 'Adventure', 'Casual', 'Free to Pla...",1643219
...,...,...
571,['Strategy'],236689747
572,"['Utilities', 'Video Production']",174788
573,['Utilities'],3311950
574,['Video Production'],354209


In [929]:
# Convertir las cadenas tipo lista en listas reales
df_playtime_mins['genres'] = df_playtime_mins['genres'].apply(ast.literal_eval)

# Crear una lista de valores únicos a partir de las listas
unique_genres = set(x for l in df_playtime_mins['genres'] for x in l)

# Crear un diccionario para almacenar las sumas de playtime_mins por género
genre_sums = {}

# Iterar a través de los valores únicos y sumar playtime_mins cuando se encuentra
for genre in unique_genres:
    genre_sums[genre] = df_playtime_mins[df_playtime_mins['genres'].apply(lambda x: genre in x)]['playtime_mins'].sum()

genre_sums

{'Action': 3074865891,
 'Video Production': 2387840,
 'Software Training': 559988,
 'Education': 412955,
 'Adventure': 898675073,
 'Design &amp; Illustration': 4075534,
 'Photo Editing': 114502,
 'Web Publishing': 2760751,
 'Free to Play': 603563327,
 'Sports': 64912685,
 'Casual': 249315152,
 'Racing': 62982260,
 'RPG': 1027848989,
 'Strategy': 650991651,
 'Massively Multiplayer': 441038277,
 'Audio Production': 545201,
 'Indie': 1475383612,
 'Early Access': 156682369,
 'Utilities': 4677356,
 'Animation &amp; Modeling': 1994171,
 'Simulation': 855261552}

In [930]:
# Crear un DataFrame a partir del diccionario
df_genre = pd.DataFrame(list(genre_sums.items()), columns=['Genre', 'Total_Playtime'])

# Ordenar el DataFrame en orden descendente por la columna "Total_Playtime" y restablecer el índice
df_genre = df_genre.sort_values(by='Total_Playtime', ascending=False).reset_index(drop=True)

# Agregar una columna con el número de posición equivalente al ranking
df_genre['Ranking'] = df_genre.reset_index(drop=False).index + 1

# agregar columnas playtime y reorganizar
df_genre['playtime_mins'] = pd.to_numeric(df_genre['Total_Playtime'], errors='coerce')

df_genre["playtime_hours"] = df_genre["playtime_mins"] / 60
df_genre["playtime_hours"] = df_genre["playtime_hours"].round(2)

df_genre["playtime_days"] = df_genre["playtime_hours"] / 24
df_genre["playtime_days"] = df_genre["playtime_days"].round(2)

df_genre["playtime_weeks"] = df_genre["playtime_days"] / 7
df_genre["playtime_weeks"] = df_genre["playtime_weeks"].round(2)

df_genre["playtime_months"] = df_genre["playtime_days"] / 30
df_genre["playtime_months"] = df_genre["playtime_months"].round(2)

df_genre["playtime_years"] = df_genre["playtime_months"] / 12
df_genre["playtime_years"] = df_genre["playtime_years"].round(2)

# Crear una columna para la busqueda de valores
df_genre['look_genre'] = df_genre['Genre'].str.lower().str.replace(" ", "")

df_genre

,Genre,Total_Playtime,Ranking,playtime_mins,playtime_hours,playtime_days,playtime_weeks,playtime_months,playtime_years
0,Action,3074865891,1,3074865891,51247764.85,2135323.54,305046.22,71177.45,5931.45
1,Indie,1475383612,2,1475383612,24589726.87,1024571.95,146367.42,34152.40,2846.03
2,RPG,1027848989,3,1027848989,17130816.48,713784.02,101969.15,23792.80,1982.73
3,Adventure,898675073,4,898675073,14977917.88,624079.91,89154.27,20802.66,1733.56
4,Simulation,855261552,5,855261552,14254359.20,593931.63,84847.38,19797.72,1649.81
5,Strategy,650991651,6,650991651,10849860.85,452077.54,64582.51,15069.25,1255.77
6,Free to Play,603563327,7,603563327,10059388.78,419141.20,59877.31,13971.37,1164.28
7,Massively Multiplayer,441038277,8,441038277,7350637.95,306276.58,43753.80,10209.22,850.77
8,Casual,249315152,9,249315152,4155252.53,173135.52,24733.65,5771.18,480.93
9,Early Access,156682369,10,156682369,2611372.82,108807.20,15543.89,3626.91,302.24


#### **3- df_genre - info**

In [931]:
count_column_types(df_genre)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,<class 'float'>,Nulos,Nulos %
0,Genre,21.0,NaN,NaN,0,0.0
1,Total_Playtime,NaN,21.0,NaN,0,0.0
2,Ranking,NaN,21.0,NaN,0,0.0
3,playtime_mins,NaN,21.0,NaN,0,0.0
4,playtime_hours,NaN,NaN,21.0,0,0.0
5,playtime_days,NaN,NaN,21.0,0,0.0
6,playtime_weeks,NaN,NaN,21.0,0,0.0
7,playtime_months,NaN,NaN,21.0,0,0.0
8,playtime_years,NaN,NaN,21.0,0,0.0


In [932]:
df_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Genre            21 non-null     object 
 1   Total_Playtime   21 non-null     int64  
 2   Ranking          21 non-null     int64  
 3   playtime_mins    21 non-null     int64  
 4   playtime_hours   21 non-null     float64
 5   playtime_days    21 non-null     float64
 6   playtime_weeks   21 non-null     float64
 7   playtime_months  21 non-null     float64
 8   playtime_years   21 non-null     float64
dtypes: float64(5), int64(3), object(1)
memory usage: 1.6+ KB


### **3- df_userforgenre**

- df_userforgenre = def userforgenre( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.
    <br>
    - df_items.user_id = (str)
    - df_items.playtime_mins = (int)
    - df_games.genres = (str / list)
    - df_items.user_url = (str)

**Observaciones y conclusiones:**

- A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
    - genre (str) = genero unico dentro de la lista total de generos
    - ranking (int) = posición de cada usuario segun la mayor cantidad de horas de juego
    - playtime_hours (int) = cantidad de horas de juego totales
    - user_id (str) = identificacion de usuario
    - user_url (str) = url de usuario

In [933]:
# nuevo dataframe
df_1 = df_items[["user_id", "user_url", "item_id", "playtime_mins"]].copy()
df_2 = df_games[["item_id", "genres"]].copy()

# Realizar un left join
df_userforgenre = df_1.merge(df_2, on='item_id', how='left')

df_userforgenre.head(5)

,user_id,user_url,item_id,playtime_mins,genres
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,6,['Action']
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,0,['Action']
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,7,['Action']
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,0,['Action']
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,0,['Action']


In [934]:
df_userforgenre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        object
 1   user_url       object
 2   item_id        object
 3   playtime_mins  int64 
 4   genres         object
dtypes: int64(1), object(4)
memory usage: 194.3+ MB


In [935]:
# Eliminar registros donde no hay ningun genero
df_userforgenre.dropna(subset=['genres'], inplace=True)

# Eliminar los registros que no tengan horas de juego (playtime_mins = 0) y reestablecer el indice
df_userforgenre = df_userforgenre[df_userforgenre["playtime_mins"] != 0].reset_index(drop=True)

# Eliminar los campos irrelevantes
df_userforgenre = df_userforgenre.drop(columns='item_id', axis=1)

# Agrupar por las columnas "user_id", "user_url" y "genres", y sumar "playtime_mins"
df_grouped = df_userforgenre.groupby(['user_id', 'user_url', 'genres'])['playtime_mins'].sum().reset_index()

df_grouped

,user_id,user_url,genres,playtime_mins
0,--000--,http://steamcommunity.com/id/--000--,"['Action', 'Adventure', 'Casual', 'Free to Pla...",4037
1,--000--,http://steamcommunity.com/id/--000--,"['Action', 'Adventure', 'Casual', 'Free to Pla...",4260
2,--000--,http://steamcommunity.com/id/--000--,"['Action', 'Adventure', 'Casual', 'Indie', 'RPG']",1597
3,--000--,http://steamcommunity.com/id/--000--,"['Action', 'Adventure', 'Casual', 'Indie']",582
4,--000--,http://steamcommunity.com/id/--000--,"['Action', 'Adventure', 'Indie', 'RPG']",1235
...,...,...,...,...
1359880,zzzmidmiss,http://steamcommunity.com/id/zzzmidmiss,"['Indie', 'Casual']",6
1359881,zzzmidmiss,http://steamcommunity.com/id/zzzmidmiss,"['Indie', 'Strategy']",114
1359882,zzzmidmiss,http://steamcommunity.com/id/zzzmidmiss,['Indie'],143
1359883,zzzmidmiss,http://steamcommunity.com/id/zzzmidmiss,['RPG'],93


In [936]:
# Crear una lista de valores únicos a partir de las listas
df_grouped['genres'] = df_grouped['genres'].apply(ast.literal_eval)
unique_genres = set(x for l in df_grouped['genres'] for x in l)
lista_genres = list(unique_genres)

lista_genres

['Action',
 'Video Production',
 'Software Training',
 'Education',
 'Adventure',
 'Design &amp; Illustration',
 'Photo Editing',
 'Web Publishing',
 'Free to Play',
 'Sports',
 'Casual',
 'Racing',
 'RPG',
 'Strategy',
 'Massively Multiplayer',
 'Audio Production',
 'Indie',
 'Early Access',
 'Utilities',
 'Animation &amp; Modeling',
 'Simulation']

In [937]:
# Crear un DataFrame para almacenar los resultados
df_userforgenre = pd.DataFrame(columns=['genre', "ranking", 'playtime_mins', 'user_id'])

# Iterar a través de las palabras únicas
for genre in lista_genres:
    # Filtrar el DataFrame para obtener los registros que contienen palabras únicas en "genres"
    filtered_df = df_grouped[df_grouped['genres'].apply(lambda x: genre in x)]

    # Eliminar la columna "genres" de filtered_df
    filtered_df = filtered_df.drop(columns="genres", axis=1)

    if genre == "Action":
        playtime_mins_1 = filtered_df.groupby(['user_id', 'user_url'])['playtime_mins'].sum().reset_index()
        playtime_mins_1['genre'] = genre
        top_records_1 = playtime_mins_1.nlargest(5, 'playtime_mins')
        top_records_1['ranking'] = top_records.reset_index(drop=False).index + 1

    # playtime_mins por usuario para ese "genre" y agregar el "genre"
    playtime_mins = filtered_df.groupby(['user_id', 'user_url'])['playtime_mins'].sum().reset_index()
    playtime_mins['genre'] = genre

    # Seleccionar los 5 registros principales para esta palabra en base a las frecuencias de "user_id"
    top_records = playtime_mins.nlargest(5, 'playtime_mins')
    top_records['ranking'] = top_records.reset_index(drop=False).index + 1

    # Agregar los registros al DataFrame final
    df_userforgenre = pd.concat([df_userforgenre, top_records])

# agregar columnas playtime y reorganizar
df_userforgenre['playtime_mins'] = pd.to_numeric(df_userforgenre['playtime_mins'], errors='coerce')

df_userforgenre["playtime_hours"] = df_userforgenre["playtime_mins"] / 60
df_userforgenre["playtime_hours"] = df_userforgenre["playtime_hours"].round(2)

df_userforgenre["playtime_days"] = df_userforgenre["playtime_hours"] / 24
df_userforgenre["playtime_days"] = df_userforgenre["playtime_days"].round(2)

df_userforgenre["playtime_weeks"] = df_userforgenre["playtime_days"] / 7
df_userforgenre["playtime_weeks"] = df_userforgenre["playtime_weeks"].round(2)

df_userforgenre["playtime_months"] = df_userforgenre["playtime_days"] / 30
df_userforgenre["playtime_months"] = df_userforgenre["playtime_months"].round(2)

df_userforgenre["playtime_years"] = df_userforgenre["playtime_months"] / 12
df_userforgenre["playtime_years"] = df_userforgenre["playtime_years"].round(2)

df_userforgenre = df_userforgenre[['genre',
                                    'ranking',
                                    'user_id',
                                    "user_url",
                                    "playtime_mins",
                                    "playtime_hours",
                                    "playtime_days",
                                    "playtime_weeks",
                                    "playtime_months",
                                    "playtime_years"]]

# Restablecer el índice del DataFrame final
df_userforgenre = df_userforgenre.reset_index(drop=True)

# Crear una columna para la busqueda de valores
df_userforgenre['look_genre'] = df_userforgenre['genre'].str.lower().str.replace(" ", "")

df_userforgenre

,genre,ranking,user_id,user_url,playtime_mins,playtime_hours,playtime_days,playtime_weeks,playtime_months,playtime_years
0,Action,1,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307,28321.78,1180.07,168.58,39.34,3.28
1,Action,2,shinomegami,http://steamcommunity.com/id/shinomegami,1580428,26340.47,1097.52,156.79,36.58,3.05
2,Action,3,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212,24270.20,1011.26,144.47,33.71,2.81
3,Action,4,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742,17762.37,740.10,105.73,24.67,2.06
4,Action,5,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193,17686.55,736.94,105.28,24.56,2.05
...,...,...,...,...,...,...,...,...,...,...
100,Simulation,1,jimmynoe,http://steamcommunity.com/id/jimmynoe,1062130,17702.17,737.59,105.37,24.59,2.05
101,Simulation,2,Evilutional,http://steamcommunity.com/id/Evilutional,684723,11412.05,475.50,67.93,15.85,1.32
102,Simulation,3,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,676540,11275.67,469.82,67.12,15.66,1.30
103,Simulation,4,tsunamitad,http://steamcommunity.com/id/tsunamitad,661309,11021.82,459.24,65.61,15.31,1.28


#### **3- df_userforgenre - info**

In [938]:
count_column_types(df_userforgenre)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'int'>,<class 'float'>,Nulos,Nulos %
0,genre,105.0,NaN,NaN,0,0.0
1,ranking,NaN,105.0,NaN,0,0.0
2,user_id,105.0,NaN,NaN,0,0.0
3,user_url,105.0,NaN,NaN,0,0.0
4,playtime_mins,NaN,105.0,NaN,0,0.0
5,playtime_hours,NaN,NaN,105.0,0,0.0
6,playtime_days,NaN,NaN,105.0,0,0.0
7,playtime_weeks,NaN,NaN,105.0,0,0.0
8,playtime_months,NaN,NaN,105.0,0,0.0
9,playtime_years,NaN,NaN,105.0,0,0.0


In [939]:
df_userforgenre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   genre            105 non-null    object 
 1   ranking          105 non-null    object 
 2   user_id          105 non-null    object 
 3   user_url         105 non-null    object 
 4   playtime_mins    105 non-null    int64  
 5   playtime_hours   105 non-null    float64
 6   playtime_days    105 non-null    float64
 7   playtime_weeks   105 non-null    float64
 8   playtime_months  105 non-null    float64
 9   playtime_years   105 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 8.3+ KB


### **3- df_developer**

- df_developer = def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.
    <br>
    - df_games.release_date (extraer el año y convertir en "release_year")(int)
    - df_games.price = (float)
    - df_games.item_id = (str)
    - df_games.developer = (str)

**Observaciones y conclusiones:**

- A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
    - developer (str) = desarrollador
    - release_year(str) = año de lanzamiento de item
    - items_count (int) = cant de items lanzados por año
    - free_count (int) = cant items price = 0 por año
    - free_percent (float) = porcentaje de free respecto al total de items del año

In [940]:
# nuevo datarame con el conteo de items por año para cada usuario

df_developer = df_games.groupby(['developer', 'release_year'])['item_id'].nunique().reset_index()
df_developer.rename(columns={'item_id': 'items_count'}, inplace=True)
df_developer

,developer,release_year,items_count
0,+7 Software,2016.0,1
1,"+Mpact Games, LLC.",2017.0,1
2,.M.Y.W.,2016.0,1
3,.ez Games,2017.0,1
4,07th Expansion,2015.0,2
...,...,...,...
14926,萌石游戏,2017.0,1
14927,高考恋爱委员会,2015.0,1
14928,"高考恋爱委员会,Days",2015.0,1
14929,"高考恋爱委员会,橘子班",2015.0,1


In [941]:
# nuevo datarame con el conteo de items free por año para cada usuario

df_developer_free = df_games[["developer", "item_id", "release_year", "price"]][df_games["price"] == 0]
df_developer_free = df_developer_free.drop(columns="price", axis=1)
df_developer_free = df_developer_free.groupby(['developer', 'release_year'])['item_id'].nunique().reset_index()
df_developer_free.rename(columns={'item_id': 'items_count_free'}, inplace=True)
df_developer_free

,developer,release_year,items_count_free
0,14 Hours Productions,2017.0,1
1,2-Volt Games,2017.0,1
2,"2Chance Projects,IIchan Eroge Team",2015.0,1
3,3D Realms,1996.0,1
4,4 I Lab,2016.0,1
...,...,...,...
1332,弘原游戏,2017.0,1
1333,拉貢拉斯找金粒,2017.0,1
1334,杭州分浪网络科技有限公司,2018.0,1
1335,深圳市云宙多媒体,2017.0,1


In [942]:
# crear llave anidada para cada dataframe
df_developer["llave"] = df_developer["developer"] + df_developer["release_year"].astype(str)
df_developer_free["llave"] = df_developer_free["developer"] + df_developer_free["release_year"].astype(str)

# nuevo dataframe
df_1 = df_developer
df_2 = df_developer_free[["llave", "items_count_free"]]

# Realizar un left join
df_developer = df_1.merge(df_2, on='llave', how='left')

df_developer

,developer,release_year,items_count,llave,items_count_free
0,+7 Software,2016.0,1,+7 Software2016.0,NaN
1,"+Mpact Games, LLC.",2017.0,1,"+Mpact Games, LLC.2017.0",NaN
2,.M.Y.W.,2016.0,1,.M.Y.W.2016.0,NaN
3,.ez Games,2017.0,1,.ez Games2017.0,NaN
4,07th Expansion,2015.0,2,07th Expansion2015.0,NaN
...,...,...,...,...,...
14926,萌石游戏,2017.0,1,萌石游戏2017.0,NaN
14927,高考恋爱委员会,2015.0,1,高考恋爱委员会2015.0,1.0
14928,"高考恋爱委员会,Days",2015.0,1,"高考恋爱委员会,Days2015.0",NaN
14929,"高考恋爱委员会,橘子班",2015.0,1,"高考恋爱委员会,橘子班2015.0",NaN


In [943]:
# eliminar campos irrelevantes
df_developer = df_developer.drop(columns="llave", axis=1)

# reemplazar nulos por 0
df_developer["items_count_free"].fillna(0, inplace=True)

# calcular la columna de porcentaje
df_developer["free_percent"] = df_developer["items_count_free"] / df_developer["items_count"] * 100
df_developer["free_percent"] = df_developer["free_percent"].round(2)

# Crear una columna para la busqueda de valores
df_developer['look_developer'] = df_developer['developer'].str.lower().str.replace(" ", "")

df_developer

,developer,release_year,items_count,items_count_free,free_percent
0,+7 Software,2016.0,1,0.0,0.0
1,"+Mpact Games, LLC.",2017.0,1,0.0,0.0
2,.M.Y.W.,2016.0,1,0.0,0.0
3,.ez Games,2017.0,1,0.0,0.0
4,07th Expansion,2015.0,2,0.0,0.0
...,...,...,...,...,...
14926,萌石游戏,2017.0,1,0.0,0.0
14927,高考恋爱委员会,2015.0,1,1.0,100.0
14928,"高考恋爱委员会,Days",2015.0,1,0.0,0.0
14929,"高考恋爱委员会,橘子班",2015.0,1,0.0,0.0


In [944]:
# prueba
df_developer[df_developer["free_percent"] > 0]

,developer,release_year,items_count,items_count_free,free_percent
32,14 Hours Productions,2017.0,2,1.0,50.00
63,2-Volt Games,2017.0,1,1.0,100.00
78,"2Chance Projects,IIchan Eroge Team",2015.0,1,1.0,100.00
140,3D Realms,1996.0,1,1.0,100.00
162,4 I Lab,2016.0,3,1.0,33.33
...,...,...,...,...,...
14906,弘原游戏,2017.0,2,1.0,50.00
14909,拉貢拉斯找金粒,2017.0,1,1.0,100.00
14910,杭州分浪网络科技有限公司,2018.0,1,1.0,100.00
14916,深圳市云宙多媒体,2017.0,1,1.0,100.00


#### **3- df_developer - info**

In [945]:
df_developer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14931 entries, 0 to 14930
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   developer         14931 non-null  object 
 1   release_year      14931 non-null  float64
 2   items_count       14931 non-null  int64  
 3   items_count_free  14931 non-null  float64
 4   free_percent      14931 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 583.4+ KB


In [946]:
count_column_types(df_developer)


filas completamente nulas:  0


,Column,<class 'str'>,<class 'float'>,<class 'int'>,Nulos,Nulos %
0,developer,14931.0,NaN,NaN,0,0.0
1,release_year,NaN,14931.0,NaN,0,0.0
2,items_count,NaN,NaN,14931.0,0,0.0
3,items_count_free,NaN,14931.0,NaN,0,0.0
4,free_percent,NaN,14931.0,NaN,0,0.0


### **3- df_sentiment_analysis**

- df_sentiment_analysis = def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
    <br>
    - df_games.release_year = (extraer el año y convertir en "release_year")(int)
    - df_games.item_id = (str)
    - df_reviews.review = (transformar la columna en "sentiment_analysis")(int)

**Observaciones y conclusiones:**

- A partir de los dataset obtener un nuevo conjunto de datos unicamente con los siguientes campos que necesitará la consulta:
    - release_year (str) = año de lanzamiento
    - sentiment_label (str) = analisis de sentimiento
    - reviews_count (int) = se refiere a cantidad de reseñas (NO tienen que ser reseñas válidas)

In [947]:
# nuevo dataframe
df_1 = df_games[["item_id", "release_year"]].copy()
df_1["item_id"] = df_1["item_id"].astype(str)

df_2 = df_reviews[["item_id", "sentiment_analysis", "sentiment_label"]].copy()
df_2["item_id"] = df_2["item_id"].astype(str)

# Realizar un left join
df_sentiment_analysis = df_1.merge(df_2, on='item_id', how='left')

In [948]:
# eliminar campos irrelevantes
df_sentiment_analysis = df_sentiment_analysis.drop(columns="item_id", axis=1)

# remover registros que no tengan analisis de sentimiento
df_sentiment_analysis.dropna(subset=['sentiment_analysis'], inplace=True)
df_sentiment_analysis.reset_index(drop=True, inplace=True)

# nuevo dataframe con valores agrupados
df_sentiment_analysis = df_sentiment_analysis.groupby(['release_year'])['sentiment_label'].value_counts().reset_index()

df_sentiment_analysis

,release_year,sentiment_label,count
0,1989.0,positive,1
1,1990.0,positive,4
2,1990.0,negative,1
3,1991.0,positive,1
4,1992.0,positive,2
...,...,...,...
73,2016.0,negative,537
74,2016.0,neutral,474
75,2017.0,positive,1299
76,2017.0,neutral,405


#### **3- df_sentiment_analysis - info**

In [949]:
df_sentiment_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   release_year     78 non-null     float64
 1   sentiment_label  78 non-null     object 
 2   count            78 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.0+ KB


In [950]:
count_column_types(df_sentiment_analysis)


filas completamente nulas:  0


,Column,<class 'float'>,<class 'str'>,<class 'int'>,Nulos,Nulos %
0,release_year,78.0,NaN,NaN,0,0.0
1,sentiment_label,NaN,78.0,NaN,0,0.0
2,count,NaN,NaN,78.0,0,0.0


### **3- df_modeloML**

- Se utilizan las columnas con la información más descriptiva de cada juego

In [993]:
# nuevo df con las columnas necesarias para el modelo
df_modelo = df_games[["item_id", "app_name", "genres", "tags"]]
df_modelo

,item_id,app_name,genres,tags
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Strategy', 'Action', 'Indie', 'Casual', 'Sim..."
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']","['Free to Play', 'Strategy', 'Indie', 'RPG', '..."
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio...","['Free to Play', 'Simulation', 'Sports', 'Casu..."
3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual']","['Action', 'Adventure', 'Casual']"
4,773570,Log Challenge,NaN,"['Action', 'Indie', 'Casual', 'Sports']"
...,...,...,...,...
32127,773640,Colony On Mars,"['Casual', 'Indie', 'Simulation', 'Strategy']","['Strategy', 'Indie', 'Casual', 'Simulation']"
32128,733530,LOGistICAL: South Africa,"['Casual', 'Indie', 'Strategy']","['Strategy', 'Indie', 'Casual']"
32129,610660,Russian Roads,"['Indie', 'Racing', 'Simulation']","['Indie', 'Simulation', 'Racing']"
32130,658870,EXIT 2 - Directions,"['Casual', 'Indie']","['Indie', 'Casual', 'Puzzle', 'Singleplayer', ..."


## **4- EXPORTAR A CSV** 

(Para funciones y modelamiento)

In [983]:
df_userdata_reduced.to_csv('eda_userdata.csv', index=False)

In [984]:
df_countreviews.to_csv('eda_countreviews.csv', index=False)

In [985]:
df_genre.to_csv('eda_genre.csv', index=False)

In [986]:
df_userforgenre.to_csv('eda_userforgenre.csv', index=False)

In [987]:
df_developer.to_csv('eda_developer.csv', index=False)

In [988]:
df_sentiment_analysis.to_csv('eda_sentiment_analysis.csv', index=False)

In [994]:
df_modelo.to_csv('eda_modelo.csv', index=False)